# IMDB
This notebook contains the first few steps of the data science pipeline on a dataset containing movies.

## Group
V2H-Groep 1: Films (IMDB)
- Niels Hoiting
- Jari Oostrom
- Yusuf Syakur

## Research questions
1. What is the correlation between the gender of actors and the popularity of the movie. How does this change overtime?
2. How can we predict a movie's genre with variables like budget, release year, actors, directors, gross, color, and any other variable?
3. To what extend can you predict the gross of a movie based on its popularity on Facebook and IMDB.
4. How accurately can we predict release years from other variables like director, gross, budget, color and any other variable?

bonus. What happens if you cluster keywords from the plot?

## Dataset
movie information with duration, genres, languages, country, budget and gross;
likes on facebook for director, main cast, total cast en the movie itself;
score on IMDB and reviews

## Step 1: Data collection
Import needed libraries. The dataset is already available.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import itertools
import json

from sklearn.cluster import KMeans

In [ ]:
df_movies = pd.read_csv('movie.csv')
df_movies

## Step 2: Data processing (Data munging)
Look at the current dataframe and their types.

In [ ]:
df_movies.describe()

In [ ]:
df_movies.dtypes

Current column order does not make sense. Order them.

In [ ]:
df_movies = df_movies[['movie_imdb_link', 'movie_title', 'imdb_score', 'title_year', 'director_name', 'director_facebook_likes', 'actor_1_name',
                      'actor_1_facebook_likes', 'actor_2_name', 'actor_2_facebook_likes', 'actor_3_name', 'actor_3_facebook_likes',
                      'cast_total_facebook_likes', 'movie_facebook_likes', 'genres', 'budget', 'gross', 'country', 'language',
                      'num_critic_for_reviews', 'num_user_for_reviews', 'num_voted_users', 'plot_keywords', 'color', 'content_rating',
                      'duration', 'aspect_ratio', 'facenumber_in_poster']]
df_movies

## Step 3: Data Cleaning

Drop overall duplicates first.

In [ ]:
print('Before removing duplicates', df_movies.shape)
df_movies = df_movies.drop_duplicates()
print('After removing duplicates:', df_movies.shape)

### 3.1 movie_imdb_link

The movie_imdb_link duplicates only differ on a few columns like likes and votes. Extract the unique identifier from the URL and remove these duplicate rows.

In [ ]:
pd.concat(gby_result for _, gby_result in df_movies.groupby("movie_imdb_link") if len(gby_result) > 1)

In [ ]:
df_movies['movie_imdb_link'] = df_movies['movie_imdb_link'].str.extract(r'(?<=title\/)(.*)(?=\/\?)', expand=False)
print('Length before removing duplicates', df_movies.shape)
df_movies = df_movies.drop_duplicates(subset='movie_imdb_link')
print('Length after removing duplicates:',df_movies.shape)

### 3.2 movie_title

Strip whitespaces from both ends for the title. Duplicate movie_title rows might be a remake or a reboot of the movie. Leave them.

In [ ]:
df_movies['movie_title'] = df_movies['movie_title'].str.strip()

### 3.3 title_year
Rows that have NaN for title_year are series/reviews, not movies. We won't need these for our analysis. CHange title_year to DateTime64 for time series analysis.

In [ ]:
df_movies.loc[df_movies['title_year'].isnull()]

In [ ]:
print('Length before removing NaN for title_year:', df_movies.shape)
df_movies = df_movies.drop(df_movies.loc[df_movies['title_year'].isnull()].index)
print('Length after removing NaN for title_year:', df_movies.shape)
df_movies['title_year'] = pd.to_datetime(df_movies['title_year'], format='%Y', errors='coerce')
df_movies

### 3.4 actor_1_name
Rows that have NaN for actor_1_name are documentaries, not movies. Remove them.

In [ ]:
df_movies.loc[df_movies['actor_1_name'].isnull()]


In [ ]:
print('Length before removing NaN for actor_1_name:', df_movies.shape)
df_movies = df_movies.drop(df_movies.loc[df_movies['actor_1_name'].isnull()].index)
print('Length after removing NaN for actor_1_name:', df_movies.shape)
df_movies

### 3.5 genres

Genres are split with an '|' delimeter. In total there are 28 unique genres. There are no NaN values. Split them and give them an own boolean column.

In [ ]:
list_genres = list(set(itertools.chain.from_iterable(df_movies.genres.str.split('|'))))
print(list_genres)

def add_genre(df, genre):
    genreConcat = 'genre_' + genre
    df_copy = df.copy()
    df_copy[genreConcat] = df_copy['genres'].str.contains(pat = genre)
    return df_copy

for genre in list_genres:
    df_movies = add_genre(df_movies, genre)

df_movies

### 3.6 plot_keywords
Remove '|' delimeter to able to use text mining (if needed).

In [ ]:
df_movies['plot_keywords'] = df_movies['plot_keywords'].str.replace('|', ' ')
df_movies['plot_keywords']

### 3.7 content_rating
Replace NaN and 'Unrated' with 'Not Rated'.

In [ ]:
print(df_movies['content_rating'].unique())

df_movies['content_rating'] = df_movies['content_rating'].str.replace('Unrated', 'Not Rated')
df_movies['content_rating'] = df_movies['content_rating'].fillna(value='Not Rated')

print(df_movies['content_rating'].unique())

### 3.8 color
All rows with NaN on color are released after 1990. Assume color is used (available since 1950s).

In [ ]:
df_movies['color'] = df_movies['color'].fillna(value='Color')
df_movies['color'].unique()

### 3.9 Remove unimportant NaN's

Remove rows that have columns with NaN values. These NaN values can't be filled in by a 'default' value. Leave budget and gross (might turn out to be too much data loss).

In [ ]:
print('Length before removing NaNs', len(df_movies))

cols_to_ignore = ['movie_imdb_link', 'budget', 'gross']
df_budget_gross = df_movies[cols_to_ignore]
df_movies = df_movies.drop(['budget', 'gross'], axis=1)

df_movies = df_movies.dropna()

print('Length after removing NaNs', len(df_movies))

df_movies = df_movies.join(df_budget_gross.set_index('movie_imdb_link'), on='movie_imdb_link')

### 3.10 Change to int64

In [ ]:
df_movies = df_movies.astype({'director_facebook_likes': 'int64',
                            'actor_1_facebook_likes': 'int64',
                            'actor_2_facebook_likes': 'int64',
                            'actor_3_facebook_likes': 'int64',
                            'cast_total_facebook_likes': 'int64',
                            'num_critic_for_reviews': 'int64',
                            'num_user_for_reviews': 'int64',
                            'num_voted_users': 'int64',
                            'duration': 'int64',
                            'facenumber_in_poster': 'int64'})

df_movies

## Step 4: Data exploration & analysis

In [ ]:
import matplotlib.pyplot as plt

Check and see if there is a correlation between budget and duration. Set a limit on budget to see a clear scatterplot. Looking at the result, there is no obvious correlation.

In [ ]:
fig = plt.figure(1, figsize=(10,10))

y_budget = df_movies[['budget']]
x_duration = df_movies[['duration']]

axScatter = plt.subplot(111)
axScatter.scatter(x_duration, y_budget)
plt.ylim(0, 300000000)
axScatter.set_title('Scatterplot between budget and duration')
axScatter.set_xlabel('Duration in minutes')
axScatter.set_ylabel('Budget in US Dollars')

We can add another dataset to see the gender of every actor.
Both datasets come from the same source: The Movie Database. 
So we'll join them on the title.

In [ ]:
df_credits = pd.read_csv('tmdb_5000_credits.csv')
df_credits = df_credits.rename(columns={'title': 'movie_title'})
df_credits

In [ ]:
movie_with_cast = pd.merge(df_movies, df_credits, how="inner", on="movie_title")
movie_with_cast

Cast is a nested field, this function will return the gender for the given cast and name.

In [ ]:

def actor_to_gender(cast, name):
    cast = json.loads(cast)
    for actor in cast:
        if name == actor['name']:
            return actor['gender']
    return 0
 
movie_with_cast['actor_1_gender'] = movie_with_cast.apply(lambda movie: actor_to_gender(movie.cast, movie.actor_1_name), axis=1)
movie_with_cast['actor_2_gender'] = movie_with_cast.apply(lambda movie: actor_to_gender(movie.cast, movie.actor_2_name), axis=1)
movie_with_cast['actor_3_gender'] = movie_with_cast.apply(lambda movie: actor_to_gender(movie.cast, movie.actor_3_name), axis=1)


In [ ]:
movie_with_cast.actor_1_gender.value_counts().plot(kind='pie', labels=['Male', 'Female', 'Unknown'])
plt.show();

In [ ]:
movie_with_cast.actor_2_gender.value_counts().plot(kind='pie', labels=['Male', 'Female', 'Unknown'])
plt.show();

In [ ]:
movie_with_cast.actor_3_gender.value_counts().plot(kind='pie', labels=['Male', 'Female', 'Unknown'])
plt.show();

The gender ratio is for the majority the same among the first three featured actors.

In [ ]:
#from pandas.plotting import scatter_matrix
#plt_scatter = scatter_matrix(df_movies[['num_critic_for_reviews', 'director_facebook_likes', 'actor_1_facebook_likes', 'gross', 'cast_total_facebook_likes', 'num_user_for_reviews', 'budget', 'imdb_score', 'movie_facebook_likes']], alpha=0.2, figsize=(9,9), diagonal='kde').view()
#plt.show()

## What happens if we cluster this dataset, leaving out the genre variable?

The goal is to see if groups that were formed as a result of clustering, make any sense when comparing it to genre. Lets make a subset of the dataframe picking up all the variables except the boolean categorical values.

In [ ]:
df_nogenre = df_movies[['movie_imdb_link', 'movie_title', 'genres', 'imdb_score', 'title_year',
       'director_name', 'director_facebook_likes', 'actor_1_name',
       'actor_1_facebook_likes', 'actor_2_name', 'actor_2_facebook_likes',
       'actor_3_name', 'actor_3_facebook_likes', 'cast_total_facebook_likes',
       'movie_facebook_likes', 'country', 'language',
       'num_critic_for_reviews', 'num_user_for_reviews', 'num_voted_users',
       'plot_keywords', 'color', 'content_rating', 'duration', 'aspect_ratio',
       'facenumber_in_poster', 'budget', 'gross']]
df_nogenre.shape

### Data pre-processing

See if clustering with k-Means using a few combinations of numerical values will create easy-to-understand groups. There are alot of categorical variables in this set, these ones will have to drop because k-Means can only use numerical variables. There is an algorithm that is similar to k-Means that is based on categorical variables. This algorithm is called k-Modes. The most important categorical variables which should indicate genre's/similar groups are actors and directors. Let's see if we can use k-Modes.

In [ ]:
print('Unique actors on 1: ', df_nogenre['actor_1_name'].unique().size)
print('Unique actors on 2: ', df_nogenre['actor_2_name'].unique().size)
print('Unique actors on 3: ', df_nogenre['actor_3_name'].unique().size)
print('Unique directors: ', df_nogenre['director_name'].unique().size)

The important categorical variables have many unique values in comparison with the amount of records. Therefore we will not use the k-Modes algorithm. Since we have facebook likes for each director and actor, we can use these numerical values for the k-Means algorithm. Let's see a sample of those likes.

In [ ]:
df_nogenre.head(5)

Hmm, that is strange. A highly appraised director such as James Cameron has zero facebook likes. Visiting his facebook page, this is not the case. Avatar is not the only one that James Cameron has directed. 

In [ ]:
df_nogenre.loc[df_nogenre['director_name'] == 'James Cameron']

All of his great works contain zero facebook likes as a director. Something went probably wrong during the retrieval of this dataset. 

A first option would be to scrape all director facebook likes properly where director_facebook_likes == 0. But this means that you would get a new timestamp of data retrieval, resulting in doing harm to the integrity of the data. Let's not do that.

Another option would be to scrape ALL of the directors facebook likes, but that is out of scope for this research.

In [ ]:
df_nogenre.loc[df_nogenre['director_facebook_likes'] == 0].shape[0]

The amount of rows with directors having zero facebook likes is 745. Dropping these would also mean dropping strong movies such as Titanic. Let's leave this variable for now.

In [ ]:
print(df_nogenre.loc[df_nogenre['actor_1_facebook_likes'] == 0].shape[0])
print(df_nogenre.loc[df_nogenre['actor_2_facebook_likes'] == 0].shape[0])
print(df_nogenre.loc[df_nogenre['actor_3_facebook_likes'] == 0].shape[0])

The amount of rows with actors having zero facebook likes totals around 80. We can drop these rows and keep the rest for our k-Means model.

In [ ]:
print('Rows before:', df_nogenre.shape[0])
df_nogenre.drop(df_nogenre.loc[df_nogenre['actor_1_facebook_likes'] == 0].index, inplace=True)
df_nogenre.drop(df_nogenre.loc[df_nogenre['actor_2_facebook_likes'] == 0].index, inplace=True)
df_nogenre.drop(df_nogenre.loc[df_nogenre['actor_3_facebook_likes'] == 0].index, inplace=True)
print('Rows left:', df_nogenre.shape[0])

The column cast_total_facebook_likes looks like a summation of all 3 actor's facebook likes. Let's test this.

In [ ]:
df_nogenre[['actor_1_facebook_likes', 'actor_2_facebook_likes', 'actor_3_facebook_likes']].sum(1)
boolean_arr = np.where(df_nogenre[['actor_1_facebook_likes', 'actor_2_facebook_likes', 'actor_3_facebook_likes']].sum(1) == df_nogenre['cast_total_facebook_likes']
                     , 'True', 'False')
truefalse, amount = np.unique(boolean_arr, return_counts=True)
np.asarray((truefalse, amount))

Apparently this is false for most of the rows. This means this column is not a result of 2 columns combined, which would interfere with creating the kMeans clusters. Let's keep this column.

Since we noted that the data might be invalid, let's see how many rows we have for each of the remaining columns being on 0.

In [ ]:
print(df_nogenre.loc[df_nogenre['movie_facebook_likes'] == 0].shape[0])
print(df_nogenre.loc[df_nogenre['num_critic_for_reviews'] == 0].shape[0])
print(df_nogenre.loc[df_nogenre['num_user_for_reviews'] == 0].shape[0])
print(df_nogenre.loc[df_nogenre['num_voted_users'] == 0].shape[0])
print(df_nogenre.loc[df_nogenre['facenumber_in_poster'] == 0].shape[0])
print(df_nogenre.loc[df_nogenre['budget'] == 0].shape[0])
print(df_nogenre.loc[df_nogenre['gross'] == 0].shape[0])
print(df_nogenre.loc[df_nogenre['cast_total_facebook_likes'] == 0].shape[0])

The amount of rows with movies having zero facebook likes is 1994. This makes sense because the dataset contains data from movies made in 1934 till 2016. Facebook was founded in 2004. Let's not use this column.

Facenumber in poster having zero makes sense, because it can contain posters not having a face on it, for example alot of thriller movies.

Since we are going with k-Means algorithm, let's drop all categorical variables as well, except for the identifiers (movie_imdb_link, movie_title, genres). We can also skip the mentioned director_facebook_likes and movie_facebook_likes column due to it's data being invalid.

In [ ]:
df_nocat = df_nogenre[['movie_imdb_link', 'movie_title', 'genres', 'imdb_score', 'title_year', 'actor_1_facebook_likes', 'actor_2_facebook_likes',
       'actor_3_facebook_likes', 'cast_total_facebook_likes', 'num_critic_for_reviews', 'num_user_for_reviews', 
       'num_voted_users', 'duration', 'aspect_ratio', 'facenumber_in_poster', 'budget', 'gross']]
df_nocat.head(5)

Because k-Means works with using the mean of a cluster, a date variable would make no sense. We can transform this variable to a numerical value. Since this dataset only contain year numbers, we only need to extract the year.

In [ ]:
df_nocat['title_year'] = pd.DatetimeIndex(df_nocat['title_year']).year
df_nocat['title_year']

To optimize the results of k-Means, let's use the Z-Score form of standardization (as opposed to min-max or decimal normalization) for all of the variables. We can only standardize if there are no NaN values, so we should drop them. The data is now ready to be clustered using the k-Means algorithm.

In [ ]:
df_nocat = df_nocat.dropna()
df_nocat[['imdb_score']] = (df_nocat[['imdb_score']]-df_nocat[['imdb_score']].mean())/df_nocat[['imdb_score']].std()
df_nocat[['actor_1_facebook_likes']] = (df_nocat[['actor_1_facebook_likes']]-df_nocat[['actor_1_facebook_likes']].mean())/df_nocat[['actor_1_facebook_likes']].std()
df_nocat[['actor_2_facebook_likes']] = (df_nocat[['actor_2_facebook_likes']]-df_nocat[['actor_2_facebook_likes']].mean())/df_nocat[['actor_2_facebook_likes']].std()
df_nocat[['actor_3_facebook_likes']] = (df_nocat[['actor_3_facebook_likes']]-df_nocat[['actor_3_facebook_likes']].mean())/df_nocat[['actor_3_facebook_likes']].std()
df_nocat[['cast_total_facebook_likes']] = (df_nocat[['cast_total_facebook_likes']]-df_nocat[['cast_total_facebook_likes']].mean())/df_nocat[['cast_total_facebook_likes']].std()
df_nocat[['num_critic_for_reviews']] = (df_nocat[['num_critic_for_reviews']]-df_nocat[['num_critic_for_reviews']].mean())/df_nocat[['num_critic_for_reviews']].std()
df_nocat[['num_user_for_reviews']] = (df_nocat[['num_user_for_reviews']]-df_nocat[['num_user_for_reviews']].mean())/df_nocat[['num_user_for_reviews']].std()
df_nocat[['num_voted_users']] = (df_nocat[['num_voted_users']]-df_nocat[['num_voted_users']].mean())/df_nocat[['num_voted_users']].std()
df_nocat[['duration']] = (df_nocat[['duration']]-df_nocat[['duration']].mean())/df_nocat[['duration']].std()
df_nocat[['aspect_ratio']] = (df_nocat[['aspect_ratio']]-df_nocat[['aspect_ratio']].mean())/df_nocat[['aspect_ratio']].std()
df_nocat[['facenumber_in_poster']] = (df_nocat[['facenumber_in_poster']]-df_nocat[['facenumber_in_poster']].mean())/df_nocat[['facenumber_in_poster']].std()
df_nocat[['budget']] = (df_nocat[['budget']]-df_nocat[['budget']].mean())/df_nocat[['budget']].std()
df_nocat[['gross']] = (df_nocat[['gross']]-df_nocat[['gross']].mean())/df_nocat[['gross']].std()
df_nocat[['title_year']] = (df_nocat[['title_year']]-df_nocat[['title_year']].mean())/df_nocat[['title_year']].std()
df_nocat

### Model building

We need to decide on a heuristic to evaluate if the clustered groups have any connection with the genre of a movie. The total amount of genres in this dataset is 28. We can say that if in every cluster there is a certain genre for >60% of that cluster, then it's safe to say that the clustering made sense (success). If it fails, then the clustering failed to make groups split by genre. In essence, we are actually testing the classification of the clustering by using the genre variable.

We need to define a function that will let us choose the hyperparameters for the model building. We are only going to toy with the amount of clusters, the list of genres and which variables will be picked up for clustering. The function will print a tuple containing the cluster, percentage of the genre and the genre itself if it passes the heuristic.

To start lets create a function that prints out a commonly found genre per cluster. We set the n_cluster to 28 (amount of genre), heuristic percentage to 60% and pick up all numeric variables.

In [ ]:
n_cluster = 28
heuristic_perc = 60
chosen_genres = list(set(itertools.chain.from_iterable(df_nocat.genres.str.split('|'))))
chosen_var = df_nocat.iloc[:,3:df_nocat.shape[1]].columns

def sum_per_cluster(df, list_var, list_genres, n_clsters, heuristic_percentage):
    print('Cluster\tPercentage\tGenre')
    allMeans = KMeans(n_clusters=n_clsters, random_state=0).fit(df.loc[:,list_var])
    df['Clusters'] = allMeans.predict(df.loc[:,list_var])
    for cluster in range(0,n_cluster):
        for genre in list_genres:
            percentage = (df.loc[df['Clusters'] == cluster].genres.str.contains(genre).sum() / df.loc[df['Clusters'] == cluster].shape[0]) * 100
            if percentage >= heuristic_percentage:
                print(cluster, '\t', np.around(percentage, decimals=2), '\t\t', genre)
                
sum_per_cluster(df_nocat, chosen_var, chosen_genres, n_cluster, heuristic_perc)

There are a few things to note about these results. There are a few clusters that contain 100% of a certain genre. This might seem ideal, but it could also be due to the small size of a cluster. We also see alot of the genre Drama. 

The result could be abit messy because:
- There is no minimum on the size of a cluster when testing the heuristic
- We set the amount of clusters to 28, because of the amount of genres. But the genres may not be even distributed and a movie may contain multiple genres.

In [ ]:
list_genres = list(set(itertools.chain.from_iterable(df_nocat.genres.str.split('|'))))
genre_percentage = list()

for genre in list_genres:
    percentage = df_nocat.loc[df_nocat['genres'].str.contains(genre)].shape[0] / df_nocat.shape[0] * 100
    genre_percentage.append((genre, np.around(percentage, decimals=2)))
    
genre_percentage.sort(key=lambda x: x[1], reverse=True)

pl_genre = [i[0] for i in genre_percentage]
pl_percentage = [i[1] for i in genre_percentage]
x_pos = np.arange(len(pl_genre)) 

plt.figure(figsize=(20,10))
plt.bar(x_pos, pl_percentage,align='center')
plt.xticks(x_pos, pl_genre, rotation=20) 
plt.ylabel('Ratio record occurences in percentage')
plt.show()

As you can see, Drama appears to be in about 50% of the dataset. The least appearing genres would be Western, Documentary and Film-Noir.

In [ ]:
print(df_nocat.loc[df_nocat['genres'].str.contains('Western')].shape[0], 'movies with Western')
print(df_nocat.loc[df_nocat['genres'].str.contains('Documentary')].shape[0], 'movies with Documentary')
print(df_nocat.loc[df_nocat['genres'].str.contains('Film-Noir')].shape[0], 'movies with Film-Noir')

Film-Noir only appears on 1 row. Western and Documentary stand on their own.

We can try again with less clusters, combining some genre's. When testing these clusters, all of the genres within the defined genre should appear in that row. Let's cap the total combined genre's at 14. These genre's will be either combined with AND (?=.*) or OR (|) in regex.

In [ ]:
new_genre_list = ['(?=.*Drama)(?=.*Romance)', '(?=.*Comedy)(?=.*Romance)', '(?=.*Action)(?=.*Adventure)', 'Sport', 'Crime',
                  '(?=.*Drama)(?=.*Thriller)', 'Horror|Mystery', '(?=.*Comedy)(?=.*Family)', '(?=.*Action)(?=.*Sci-Fi)'
                  , 'War|History|Western', 'Animation', 'Music|Musical', 'Documentary|Biography', '(?=.*Action)(?=.*Fantasy)']

We tried to combine the higher occuring genres alot, so that it evens out. Let's see what the current distribution between the combined genre's are.

In [ ]:
genre_percentage = list()

for genre in new_genre_list:
    percentage = df_nocat.loc[df_nocat['genres'].str.contains(genre, regex=True)].shape[0] / df_nocat.shape[0] * 100
    genre_percentage.append((genre, np.around(percentage, decimals=2)))
    
genre_percentage.sort(key=lambda x: x[1], reverse=True)

pl_genre = [i[0] for i in genre_percentage]
pl_percentage = [i[1] for i in genre_percentage]
x_pos = np.arange(len(pl_genre)) 

plt.figure(figsize=(20,10))
plt.bar(x_pos, pl_percentage,align='center')
plt.xticks(x_pos, pl_genre, rotation=40) 
plt.ylabel('Ratio record occurences in percentage')
plt.show()

The new genre balances out somewhat better than before.

We need to add the new heuristic (size of cluster) and test cases (combined genre) to the function. For the size of a cluster, let's pick a minimum of 20. We also lower the percentage range from 60 to 25 to be able to print results.

In [ ]:
#Drop last Cluster
df_nocat = df_nocat.iloc[:, :-1]

n_cluster = 14
heuristic_perc = 25
chosen_genres = new_genre_list
chosen_var = df_nocat.iloc[:,3:df_nocat.shape[1]].columns

def sum_per_cluster(df, list_var, list_genres, n_clsters, heuristic_percentage):
    print('Cluster\tPercentage\tGenre')
    allMeans = KMeans(n_clusters=n_clsters, random_state=0).fit(df.loc[:,list_var])
    df['Clusters'] = allMeans.predict(df.loc[:,list_var])
    for cluster in range(0,n_cluster):
        if df.loc[df['Clusters'] == cluster].shape[0] < 20:
            continue
        for genre in list_genres:
            percentage = (df.loc[df['Clusters'] == cluster].genres.str.contains(genre, regex=True).sum() / df.loc[df['Clusters'] == cluster].shape[0]) * 100
            if percentage >= heuristic_percentage:
                print(cluster, '\t', np.around(percentage, decimals=2), '\t\t', genre)
                
sum_per_cluster(df_nocat, chosen_var, chosen_genres, n_cluster, heuristic_perc)

Hmm looks like the results are even worse when combining genres, because the percentages are alot lower. How about if we pick different variables? Let's keep all social media stuff and remove properties from the movie itself (and vica versa)

In [ ]:
chosen_var = ['imdb_score', 'actor_1_facebook_likes',
       'actor_2_facebook_likes', 'actor_3_facebook_likes',
       'cast_total_facebook_likes', 'num_critic_for_reviews',
       'num_user_for_reviews', 'num_voted_users']

sum_per_cluster(df_nocat, chosen_var, chosen_genres, n_cluster, heuristic_perc)

In [ ]:
chosen_var = ['title_year', 'duration', 'aspect_ratio',
       'facenumber_in_poster', 'budget', 'gross']

sum_per_cluster(df_nocat, chosen_var, chosen_genres, n_cluster, heuristic_perc)

### Conclusion

In the latter model we see somewhat more distinct genres. However the percentages are still not good enough to say that kMeans can cluster this set into different groups of genres. To give it a better picture, we can visualize this by plotting the largest 3 clusters and their genre distribution.

In [ ]:
df_nocat.Clusters.value_counts()

In [ ]:
df8 = df_nocat.loc[df_nocat['Clusters'] == 8]
df12 = df_nocat.loc[df_nocat['Clusters'] == 12]
df3 = df_nocat.loc[df_nocat['Clusters'] == 3]

genre_percentage8 = list()
genre_percentage12 = list()
genre_percentage3 = list()

for genre in new_genre_list:
    percentage8 = df8.loc[df8['genres'].str.contains(genre, regex=True)].shape[0] / df8.shape[0] * 100
    percentage12 = df12.loc[df12['genres'].str.contains(genre, regex=True)].shape[0] / df12.shape[0] * 100
    percentage3 = df3.loc[df3['genres'].str.contains(genre, regex=True)].shape[0] / df3.shape[0] * 100
    genre_percentage8.append((genre, np.around(percentage8, decimals=2)))
    genre_percentage12.append((genre, np.around(percentage12, decimals=2)))
    genre_percentage3.append((genre, np.around(percentage3, decimals=2)))

barwidth = 0.25    

pl_genre8 = [i[0] for i in genre_percentage8]
pl_percentage8 = [i[1] for i in genre_percentage8]
pl_percentage12 = [i[1] for i in genre_percentage12]
pl_percentage3 = [i[1] for i in genre_percentage3]
x_pos8 = np.arange(len(pl_genre))
x_pos12 = [x + barwidth for x in x_pos8]
x_pos3 = [x + barwidth for x in x_pos12]

plt.figure(figsize=(20,10))
plt.bar(x_pos8, pl_percentage8, edgecolor='white', width=barwidth)
plt.bar(x_pos12, pl_percentage12,edgecolor='white', width=barwidth)
plt.bar(x_pos3, pl_percentage3, edgecolor='white', width=barwidth)
plt.xticks([r + barwidth for r in range(len(pl_genre))], pl_genre8, rotation=40) 
plt.ylabel('Ratio record occurences in percentage')
plt.ylim(0, 70)
plt.legend(['8', '12', '3'], title='Clusters')
plt.hlines(60, xmin=0, xmax=14, linestyle='dashed', colors='red')
plt.show()

There isn't really a good distinction between the large clusters themselves, except for green maybe which outshines on War|History|Western and Documentary|Biography. **But none of the distributions are close to the 60% heuristic**. Therefor it is safe to say that clustering this dataset with kMeans **cannot** result into groups of genres.